In [1]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2


In [2]:

import numpy as np
print(np.__version__)
from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle

1.26.4


c:\Users\User\Desktop\Диплом\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

In [4]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
 # тут заменить на нужную папку
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


In [5]:

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.5, max_chunk_size=1000)
algorithm.execute(visualize=True)

Splitting text into the sentences


442it [00:00, 26173.32it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 442/442 [01:42<00:00,  4.31it/s]


Adding chunks to the storage


100%|██████████| 192/192 [00:01<00:00, 98.80it/s] 


1

In [6]:
import random
index = chunks.get_indexes()[random.randint(1, 50)]
print(len(chunks[index]))
chunks[index]

987


'2.5 Conclusion 19\n2.5 Conclusion\nThese works, including the notable "Think Before You Speak" [19] paper,\nhave introduced methods such as pause tokens to address computational ineffi-\nciencies. These studies have laid a foundation by showing that allowing models\nto ’pause’ can enhance their processing capabilities. Building on these foundations, this thesis introduces a method that advances\nthe concept of pause tokens by eliminating the need for fine-tuning and changing\nthe way of detection of the need to pause. Unlike the methods shown in the\nexisting works, which typically require model retraining or changing the way\nit’s prompted, this research proposes an on-the-fly inference-time mechanic that\nidentifies complex areas of the text in real time. When such an area is detected,\nthe model is forced to pause for multiple steps, iteratively refining its next output. Duringthesepauses,themodel’stemperatureislowered,sharpeningitsfocusand\nincreasing the certainty of its outputs.

In [7]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = L2RAMVectorIndex(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 192/192 [01:08<00:00,  2.79it/s]


In [8]:
query = 'What is LLM'
query_vector = embedder.embed(text=query)
# print(query_vector)
vector_answer = vector_index.query(
    query_vector, top_k=5
)
print(vector_answer)

ValidationError: 1 validation error for QueryResult
score
  Field required [type=missing, input_value={'distance': 0.42413747, ...4ef1-a79c-a3951cd8d9af'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
from mirage.inference.quorums.RusVectoresQuorum import RusVectoresQuorum


quorum = RusVectoresQuorum(
    chunk_storage=chunks, 
    path_to_model='mirage\inference\quorums\model.bin',
    POS_thresholds={"ADJ": 0.5, "VERB": 0.65},
    visualize=True,
    max_combinations=25)
fulltext_search_answer = quorum.query(query)

<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
C:\Users\User\AppData\Local\Temp\ipykernel_17524\3838183163.py:6: SyntaxWarning: invalid escape sequence '\i'
  path_to_model='mirage\inference\quorums\model.bin',
C:\Users\User\AppData\Local\Temp\ipykernel_17524\3838183163.py:6: SyntaxWarning: invalid escape sequence '\i'
  path_to_model='mirage\inference\quorums\model.bin',


FileNotFoundError: [Errno 2] No such file or directory: 'mirage\\inference\\quorums\\model.bin'

In [ ]:
from mirage.inference.rerankers.LinearCombinationReranker import LinearCombinationReranker

rank_fusion = LinearCombinationReranker(fulltext_score_weight=0.25, vector_score_weight=1)
ranked_answer = rank_fusion(fulltext_search_answer, vector_answer)

NameError: name 'fulltext_search_answer' is not defined

In [ ]:
chunks.get_texts_for_search_results(ranked_answer)

In [20]:
from mirage.inference.prompters.APILLM import LLM
from mirage.inference.prompters.LLMGroq import GroqLLM
llm = GroqLLM()
llm.do_request(query="What this text about?",#query,
               chunk_storage=chunks,
            #    indexes=[element.chunk_storage_key for element in ranked_answer],
               indexes=chunks.get_indexes()[9:12],
               prompt_template=None)

This text is about the theoretical framework and methodology of Large Language Models (LLMs) in natural language processing (NLP). It discusses the softmax function, temperature parameter, and its effect on the generated text, as well as the Transformer model and its self-attention system. Additionally, it touches on the topic of fine-tuning LLMs to follow specific instructions and proposes two mechanics to handle pause tokens.
